In [1]:
import os
import glob
import numpy as np
import scipy
import astropy.table
import astropy.io.fits
import astropy.units
import random
import copy

import matplotlib.pyplot as plt

import lezargus


%matplotlib Qt

ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
ToDoError - Uncertainty

In [2]:
lezargus.initialize.initialize_logging_outputs()
# Making sure that the output directory exists.
os.makedirs("./products/spectre_calibrations/", exist_ok=True)

In [3]:
def read_dat_arc_files(filename: str) -> tuple[list, list]:
    """Read in a text/dat based arc data file."""
    loaded_data = np.genfromtxt(filename, dtype=float, comments="#")
    raw_wavelength, raw_flux = loaded_data.T

    # We want to ensure that no NaNs screw things up so we just interpolate
    # in the middle in NaN cases.
    wavelength = raw_wavelength
    interpolator = lezargus.library.interpolate.Linear1DInterpolate(
        x=raw_wavelength, v=raw_flux, extrapolate=False
    )
    flux = interpolator(wavelength)

    return wavelength, flux


def read_spex_arc_files(filename: str) -> tuple[list, list]:
    """Read in the SpeX arc data."""
    with astropy.io.fits.open(filename) as hdul:
        hdu = hdul["PRIMARY"]
        hdu_data = hdu.data
        raw_wavelength, raw_flux, raw_error, __ = hdu_data

    # We want to ensure that no NaNs screw things up so we just interpolate
    # in the middle in NaN cases.
    wavelength = raw_wavelength
    interpolator = lezargus.library.interpolate.Linear1DInterpolate(
        x=raw_wavelength, v=raw_flux, extrapolate=False
    )
    flux = interpolator(wavelength)

    # All done.
    return wavelength, flux

## Simulation Th Ar Lamps

In [4]:
thar_lamp_filename = "./base/spectre_calibrations/thar_spec_MM201006_r2000.dat"
thar_wave, thar_flux = read_dat_arc_files(filename=thar_lamp_filename)

# Converting the wavelength and data.
thar_wave_si = lezargus.library.conversion.convert_units(
    value=thar_wave, value_unit="um", result_unit="m"
)
thar_flux_si = lezargus.library.conversion.convert_units(
    value=thar_flux, value_unit="count s^-1", result_unit="count s^-1"
)

thar_wave_column = astropy.table.Column(
    thar_wave_si, name="wavelength", unit="m", dtype=float
)
thar_flux_column = astropy.table.Column(
    thar_flux_si, name="flux", unit="count s^-1", dtype=float
)

# Readout.
thar_table_columns = [thar_wave_column, thar_flux_column]
thar_arclamp_table = astropy.table.Table(thar_table_columns)
# Too much precision makes unnessary large file sizes.
thar_out_filename = f"./products/spectre_calibrations/visible_thar_arclamp.dat"
thar_arclamp_table.write(
    thar_out_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in thar_arclamp_table.keys()},
    overwrite=True,
)

## Simulation Hg Ar Arcs

In [5]:
hgar_lamp_filename = (
    "./base/spectre_calibrations/hgar_arcspectrum_0p4_1p0um.dat"
)
hgar_wave, hgar_flux = read_dat_arc_files(filename=hgar_lamp_filename)

# Converting the wavelength and data.
hgar_wave_si = lezargus.library.conversion.convert_units(
    value=hgar_wave, value_unit="um", result_unit="m"
)
hgar_flux_si = lezargus.library.conversion.convert_units(
    value=hgar_flux, value_unit="count s^-1", result_unit="count s^-1"
)

# Small delta to match up with SXD/LXDs level arc lamp levels.
hgar_flux_si = hgar_flux_si * 1 * 1e4

hgar_wave_column = astropy.table.Column(
    hgar_wave_si, name="wavelength", unit="m", dtype=float
)
hgar_flux_column = astropy.table.Column(
    hgar_flux_si, name="flux", unit="count s^-1", dtype=float
)

# Readout.
hgar_table_columns = [hgar_wave_column, hgar_flux_column]
hgar_arclamp_table = astropy.table.Table(hgar_table_columns)
# Too much precision makes unnessary large file sizes.
hgar_out_filename = f"./products/spectre_calibrations/visible_hgar_arclamp.dat"
hgar_arclamp_table.write(
    hgar_out_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in hgar_arclamp_table.keys()},
    overwrite=True,
)

## Simulation SXD Arcs

In [6]:
spex_sxd_filename = "./base/spectre_calibrations/spex_sxd_0p3slit_arc.fits"
sxd_wave, sxd_flux = read_spex_arc_files(filename=spex_sxd_filename)

# Converting the wavelength and data.
sxd_wave_si = lezargus.library.conversion.convert_units(
    value=sxd_wave, value_unit="um", result_unit="m"
)
sxd_flux_si = lezargus.library.conversion.convert_units(
    value=sxd_flux, value_unit="count s^-1", result_unit="count s^-1"
)

sxd_wave_column = astropy.table.Column(
    sxd_wave_si, name="wavelength", unit="m", dtype=float
)
sxd_flux_column = astropy.table.Column(
    sxd_flux_si, name="flux", unit="count s^-1", dtype=float
)

# Readout.
sxd_table_columns = [sxd_wave_column, sxd_flux_column]
sxd_arclamp_table = astropy.table.Table(sxd_table_columns)
# Too much precision makes unnessary large file sizes.
sxd_out_filename = (
    f"./products/spectre_calibrations/nearir_spex_sxd_arclamp.dat"
)
sxd_arclamp_table.write(
    sxd_out_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in sxd_arclamp_table.keys()},
    overwrite=True,
)

 ############################## Xspextool History ############################## [astropy.io.fits.card]
 ############################# Xmergeorders History ############################ [astropy.io.fits.card]


## Simulation LXDS Arcs

In [7]:
spex_lxds_filename = (
    "./base/spectre_calibrations/spex_lxdshort_0p8slit_arc.fits"
)
lxds_wave, lxds_flux = read_spex_arc_files(filename=spex_lxds_filename)

# Converting the wavelength and data.
lxds_wave_si = lezargus.library.conversion.convert_units(
    value=lxds_wave, value_unit="um", result_unit="m"
)
lxds_flux_si = lezargus.library.conversion.convert_units(
    value=lxds_flux, value_unit="count s^-1", result_unit="count s^-1"
)

lxds_wave_column = astropy.table.Column(
    lxds_wave_si, name="wavelength", unit="m", dtype=float
)
lxds_flux_column = astropy.table.Column(
    lxds_flux_si, name="flux", unit="count s^-1", dtype=float
)

# Readout.
lxds_table_columns = [lxds_wave_column, lxds_flux_column]
lxds_arclamp_table = astropy.table.Table(lxds_table_columns)
# Too much precision makes unnessary large file sizes.
lxds_out_filename = (
    f"./products/spectre_calibrations/midir_spex_lxds_arclamp.dat"
)
lxds_arclamp_table.write(
    lxds_out_filename,
    format="ascii.mrt",
    formats={keydex: "%.7e" for keydex in lxds_arclamp_table.keys()},
    overwrite=True,
)

 ############################## Xspextool History ############################## [astropy.io.fits.card]
 ############################# Xmergeorders History ############################ [astropy.io.fits.card]


## Simulation SPECTRE Wavelength Solution

In [8]:
# Visible Channel

In [4]:
visible_simulate = lezargus.simulator.SpectreSimulator.from_blackbody(
    wavelength=np.linspace(0.4, 0.85, 1000) * 1e-6,
    blackbody_temperature=3000,
    magnitude=10,
    photometric_filter=lezargus.data.FILTER_JOHNSON_V,
    spectral_scale=3.1e-8,
    channel="visible",
    exposure_time=60.0,
    spatial_oversample=2,
    zenith_angle=np.deg2rad(60),
    parallactic_angle=np.deg2rad(45),
)
# Arc lamp in.
visible_simulate.arc_lamp_in = True
visible_arclamp_image = copy.deepcopy(visible_simulate.at_scattered_light)
# And the flat lamp in.
visible_simulate.clear_cache()
visible_simulate.arc_lamp_in = False
visible_simulate.flat_lamp_in = True
visible_flatlamp_image = copy.deepcopy(visible_simulate.at_scattered_light)

# Noise adaption...
visible_arclamp_image.data += (
    np.random.random(size=visible_arclamp_image.data.shape) / 1e6
)

# The retriever...
visible_retriever = lezargus.pipeline.SpectreRetrieval(
    flat_image=visible_arclamp_image,
    arc_image=visible_arclamp_image,
    channel="visible",
    image=None,
)

[Lezargus] 2026-02-12T00:39:17    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T00:39:17    ERROR -- ToDoError - Uncertainty values on integrations need to be done.
[Lezargus] 2026-02-12T00:39:17    ERROR -- ToDoError - Uncertainty values on integrations need to be done.
[Lezargus] 2026-02-12T00:39:18    ERROR -- ToDoError - Simulated arc lamps are broken into 3 channels; but there is only one arc lamp. A full spectrum one should be used.
[Lezargus] 2026-02-12T00:39:18  WARNING -- AccuracyWarning - Interpolation is attempted at a wavelength beyond the domain of wavelengths of this spectrum. The overlap fraction is 0.7501236872812133.
[Lezargus] 2026-02-12T00:39:18    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargus] 2026-02-12T00:39:22    ERROR -- ToDoError - Flux conserving interpolation is needed to be implemented; defaulting to Linear.
[Lezargu

In [5]:
slice_index = 1

# Extract the wavelength slice...
wavelength_slice = visible_retriever.retrieve_slice(
    image=visible_arclamp_image,
    slice_=slice_index,
    buffer_width=7,
    rotate=False,
)
wavelength_slice_array = wavelength_slice.data
# And create the arclamp spectrum, though this is an array...
arclamp_spectrum = np.mean(wavelength_slice_array, axis=1)

# This is needed for the next step, determining the wavelength solution truth
# manually...
visible_simulation_arclamp = lezargus.data.SPECTRE_ARCLAMP_SIMULATION_VISIBLE
smoothing_kernel = lezargus.library.convolution.kernel_1d_gaussian(100, 10)
visible_simulation_arclamp_convolved = visible_simulation_arclamp.convolve(
    kernel=smoothing_kernel
)

[Lezargus] 2026-02-12T00:41:20    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T00:41:20    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T00:41:20    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T00:41:20    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T00:41:20    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T00:41:20    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T00:41:20    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.
[Lezargus] 2026-02-12T00:41:20    ERROR -- ToDoError - Trim slice rebinning needs to be done. Doing a bad interpolation job.


In [72]:
plt.figure()
plt.title("Visible Arclamp")
plt.plot(visible_simulation_arclamp.wavelength, visible_simulation_arclamp.data)
plt.show()
plt.figure()
plt.title("Visible Arclamp Convolved")
plt.plot(
    visible_simulation_arclamp_convolved.wavelength,
    visible_simulation_arclamp_convolved.data,
)
plt.show()
plt.figure()
plt.title("Simulation Data")
plt.plot(arclamp_spectrum)
plt.show()

visible_translation_table = {}